<a href="https://colab.research.google.com/github/bacoco/LLM-Finetuning/blob/main/adalflow_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 Welcome to AdalFlow!
## The Library to Build and to Auto-optimize Any LLM Task Pipeline

Thanks for trying us out, we're here to provide you with the best LLM application development experience you can dream of 😊 any questions or concerns you may have, [come talk to us on discord,](https://discord.gg/ezzszrRZvT) we're always here to help!

# What is AdalFlow?

*AdalFlow* helps developers build and optimize *Retriever-Agent-Generator* pipelines.
Embracing similar design pattern to *PyTorch*, AdalFlow is *light*, *modular*, and *robust*, with a 100% readable codebase.

# Quick Links

Github repo: https://github.com/SylphAI-Inc/AdalFlow

Full Tutorials: https://adalflow.sylph.ai/index.html#.


# Installation

1. Use `pip` to install the `adalflow` Python package. We will need `openai`, `groq`, and `faiss`(cpu version) from the extra packages.

  ```bash
  pip install adalflow[openai,groq,faiss-cpu]
  ```
2. Setup  `openai` and `groq` API key in the environment variables

In [ ]:
from IPython.display import clear_output

!pip install -U adalflow[openai,groq,faiss-cpu]

clear_output()

## Set Environment Variables

Run the following code and pass your api key.

Note: for normal `.py` projects, follow our [official installation guide](https://lightrag.sylph.ai/get_started/installation.html).

*Go to [OpenAI](https://platform.openai.com/docs/introduction) and [Groq](https://console.groq.com/docs/) to get API keys if you don't already have.*

In [ ]:
import os

from getpass import getpass

# Prompt user to enter their API keys securely
openai_api_key = getpass("Please enter your OpenAI API key: ")
groq_api_key = getpass("Please enter your GROQ API key: ")


# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['GROQ_API_KEY'] = groq_api_key

print("API keys have been set.")

Please enter your OpenAI API key: ··········
Please enter your GROQ API key: ··········
API keys have been set.



# 😇 Create Your First ChatBot

We will start with a single turn chatbot which will explain concepts with ``explanation`` and ``example``. To achieve this, we will build a simple pipeline to get the **structured output** as ``QAOutput``.


##Well-designed Base Classes


We will use this use case to demonstrate how to leverage our two and only powerful base classes: `Component` as building blocks for the pipeline and `DataClass` to ease the data interaction with LLMs.




In [ ]:
# Prepare data and template [jinja2 syntax]

from dataclasses import dataclass, field
from typing import Dict
import adalflow as adal

from adalflow.core import Component, Generator, DataClass, ModelClient
from adalflow.components.model_client import GroqAPIClient
from adalflow.components.output_parsers import JsonOutputParser

@dataclass
class QAOutput(DataClass):
    explanation: str = field(
        metadata={"desc": "A brief explanation of the concept in one sentence."}
    )
    example: str = field(metadata={"desc": "An example of the concept in a sentence."})



qa_template = r"""<SYS>
You are a helpful assistant.
<OUTPUT_FORMAT>
{{output_format_str}}
</OUTPUT_FORMAT>
</SYS>
User: {{input_str}}
You:"""

In [ ]:
# Create the task pipeline

class QA(adal.Component):
    def __init__(self, model_client: ModelClient, model_kwargs: Dict):
        super().__init__()

        parser = JsonOutputParser(data_class=QAOutput, return_data_class=True)
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
            prompt_kwargs={"output_format_str": parser.format_instructions()},
            output_processors=parser,
        )

    def call(self, query: str):
        return self.generator.call({"input_str": query})

    async def acall(self, query: str):
        return await self.generator.acall({"input_str": query})

## Clear Pipeline Structure

Simply by using `print(qa)`, you can see the pipeline structure, which helps users understand any LLM workflow quickly, especially when the pipeline is complicated.



In [ ]:
# Instantiate the QA class

qa = QA(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
)

print(qa)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
QA(
  (generator): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, 
    (prompt): Prompt(
      template: <SYS>
      You are a helpful assistant.
      <OUTPUT_FORMAT>
      {{output_format_str}}
      </OUTPUT_FORMAT>
      </SYS>
      User: {{input_str}}
      You:, prompt_kwargs: {'output_format_str': 'Your output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",\n    "example": "An example of the concept in a sentence. (str) (required)"\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.'}, prompt_variables:

In [ ]:
# call the qa and check the output

qa("What is LLM?")

GeneratorOutput(id=None, data=QAOutput(explanation='LLM stands for Large Language Model, a type of AI model designed to process and generate large amounts of natural language data.', example='For instance, LLMs are used in chatbots and virtual assistants to understand and respond to user queries.'), error=None, usage=CompletionUsage(completion_tokens=63, prompt_tokens=172, total_tokens=235), raw_response='```\n{\n    "explanation": "LLM stands for Large Language Model, a type of AI model designed to process and generate large amounts of natural language data.",\n    "example": "For instance, LLMs are used in chatbots and virtual assistants to understand and respond to user queries."\n}', metadata=None)

In [ ]:
# display the prompt only

qa.generator.print_prompt(
        output_format_str=qa.generator.output_processors.format_instructions(),
        input_str="What is LLM?",
)

Prompt:
______________________
<SYS>
You are a helpful assistant.
<OUTPUT_FORMAT>
Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.
-Follow the JSON formatting conventions.
</OUTPUT_FORMAT>
</SYS>
User: What is LLM?
You:


'<SYS>\nYou are a helpful assistant.\n<OUTPUT_FORMAT>\nYour output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",\n    "example": "An example of the concept in a sentence. (str) (required)"\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.\n</OUTPUT_FORMAT>\n</SYS>\nUser: What is LLM?\nYou:'

## Model-Agnostic

You can switch to any model simply by using a different model_client (provider) and model_kwargs.
Let's use OpenAI's gpt-3.5-turbo model on the same pipeline.

In [ ]:
from adalflow.components.model_client import OpenAIClient

qa_with_gpt = QA(
    model_client=OpenAIClient(),
    model_kwargs={"model": "gpt-3.5-turbo"}
)

qa_with_gpt("What is LLM?")

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db


GeneratorOutput(id=None, data=QAOutput(explanation='LLM stands for Large Language Model, which refers to a type of natural language processing model with a high number of parameters.', example='GPT-3 is an example of an LLM that has 175 billion parameters.'), error=None, usage=CompletionUsage(completion_tokens=59, prompt_tokens=169, total_tokens=228), raw_response='```\n{\n    "explanation": "LLM stands for Large Language Model, which refers to a type of natural language processing model with a high number of parameters.",\n    "example": "GPT-3 is an example of an LLM that has 175 billion parameters."\n}\n```', metadata=None)

# 🤗 Create First Retrieval-augmented Generation(RAG) pipeline

We will use local data base `LocalDB` and `core.data_process` to create a data processing pipeline. This data pipeline will split documents into chunks and work with `LocalDB` to persis the transformed/processed documents in local file `index.faiss` (pickle format).

## Use Config

We will put all configurations together in `config` as dict.

In [ ]:
configs = {
    "embedder": {
        "batch_size": 100,
        "model_kwargs": {
            "model": "text-embedding-3-small",
            "dimensions": 256,
            "encoding_format": "float",
        },
    },
    "retriever": {
        "top_k": 2,
    },
    "generator": {
        "model": "gpt-3.5-turbo",
        "temperature": 0.3,
        "stream": False,
    },
    "text_splitter": {
        "split_by": "word",
        "chunk_size": 400,
        "chunk_overlap": 200,
    },
}

## Prepare data pipeline

Data pipeline requires a sequence of `Document` as inputs.

In [ ]:
from adalflow.components.data_process import (
    RetrieverOutputToContextStr,
    ToEmbeddings,
    TextSplitter,
)
from adalflow.core.container import Sequential

from adalflow.core.types import Document, ModelClientType


def prepare_data_pipeline():
    splitter = TextSplitter(**configs["text_splitter"])
    embedder = adal.Embedder(
        model_client=ModelClientType.OPENAI(),
        model_kwargs=configs["embedder"]["model_kwargs"],
    )
    embedder_transformer = ToEmbeddings(
        embedder=embedder, batch_size=configs["embedder"]["batch_size"]
    )
    data_transformer = Sequential(splitter, embedder_transformer)
    return data_transformer

In [ ]:
data_transformer = prepare_data_pipeline()
data_transformer

Sequential(
  (0): TextSplitter(split_by=word, chunk_size=400, chunk_overlap=200)
  (1): ToEmbeddings(
    batch_size=100
    (embedder): Embedder(
      model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
      (model_client): OpenAIClient()
    )
    (batch_embedder): BatchEmbedder(
      (embedder): Embedder(
        model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
        (model_client): OpenAIClient()
      )
    )
  )
)

In [ ]:
# Prepare documents for data transformer

doc1 = Document(
        meta_data={"title": "Li Yin's profile"},
        text="My name is Li Yin, I love rock climbing" + "lots of nonsense text" * 500,
        id="doc1",
)
doc2 = Document(
    meta_data={"title": "Interviewing Li Yin"},
    text="lots of more nonsense text" * 250
    + "Li Yin is an AI researcher and a software engineer"
    + "lots of more nonsense text" * 250,
    id="doc2",
)

doc1

Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None)

In [ ]:
# transform the data

transformed_documents = data_transformer([doc1, doc2])

Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]
Adding embeddings to documents from batch: 1it [00:00, 6842.26it/s]


## Transformed documents

From the following visualization, we will see `doc1` is splitted into 7 chunks and `doc2` is splitted into 10 chunks. We get this relation from reading the `transformed_documents`, the `parent_doc_id` field.

Note: For `text` and `vector`, we dont show the full text or the full vector as it is rather long. You can access each field directly to visualize the full value

In [ ]:
# visualize the transformed data
transformed_documents

[Document(id=4ecdc0c6-9786-4634-b6dd-4c0d41eb1e0a, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None),
 Document(id=19db21df-9565-4be8-8073-a935d493b20c, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=1, score=None),
 Document(id=1561f534-59c6-4dc8-9da9-3a4304940e6a, text='nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlot...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=2, score=None),
 Document(id=f6eb5833-51cf-4290-b764-b230decad3fb, text='of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense text...', meta_data={'title': "Li Yin's profile"}, v

## Use LocalDB

We will use localdb to manage the `documents`, `transformers`, and the persistance of the transformed documents. This resembles more of the production environment where the embeddings and documents are often handled in data base and can be reused to save cost.

In [ ]:
from typing import List
from adalflow.core.db import LocalDB


def prepare_database_with_index(docs: List[Document], index_path: str = "index.faiss"):
    if os.path.exists(index_path):
        return None
    db = LocalDB()
    db.load(docs)
    data_transformer = prepare_data_pipeline()
    db.transform(data_transformer, key="data_transformer")
    # store
    db.save_state(index_path)
    print(db)

In [ ]:
# prepare the database for retriever

prepare_database_with_index([doc1, doc2], index_path="index.faiss")

Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]
Adding embeddings to documents from batch: 1it [00:00, 6482.70it/s]

LocalDB(name='LocalDB', items=[Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=doc2, text='lots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonsense ...', meta_data={'title': 'Interviewing Li Yin'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_items={'data_transformer': [Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=55ea86ea-175b-4a6b-ac4e-df303cb8440f, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vect

## Load from file

LocalDB `save_state` not only persist the transformed documents, but also the `data_transformer`.

This is really helpful as your retriever needs to have a matching `embedder` to embed the string query. Saving the transformer lets you verify and know what embedder you need to pass to Retriever.

In [ ]:
# test the database loading

db = LocalDB.load_state("index.faiss")
db

LocalDB(name='LocalDB', items=[Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=doc2, text='lots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonsense ...', meta_data={'title': 'Interviewing Li Yin'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_items={'data_transformer': [Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=55ea86ea-175b-4a6b-ac4e-df303cb8440f, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vect

In [ ]:
# test data fetching from local db

db.get_transformed_data("data_transformer")

[Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None),
 Document(id=55ea86ea-175b-4a6b-ac4e-df303cb8440f, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=1, score=None),
 Document(id=464775cb-637a-43bc-be85-908a7a35e9c4, text='nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlot...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=2, score=None),
 Document(id=0447254c-a093-4e56-9ad1-a7a8730742d0, text='of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense text...', meta_data={'title': "Li Yin's profile"}, v

## RAG pipeline

Now, we will create a RAG pipeline, it consists of:
* db (we will load from index_path), we will use `data_transformer` as the key to load the transformed documents.
* `FAISSRetriever` which will use embeddings to perform semantic search, and return similarity score in range [0, 1].
* `RetrieverOutputToContextStr`: this will convert the retrieved documents to a single str.
* `Generator`: we will use a simple `JsonParser` to output a dict with field `answer`.

In [ ]:
from typing import Optional, Any

from adalflow.core.string_parser import JsonParser
from adalflow.components.retriever.faiss_retriever import FAISSRetriever


rag_prompt_task_desc = r"""
You are a helpful assistant.

Your task is to answer the query that may or may not come with context information.
When context is provided, you should stick to the context and less on your prior knowledge to answer the query.

Output JSON format:
{
    "answer": "The answer to the query",
}"""


class RAG(Component):

    def __init__(self, index_path: str = "index.faiss"):
        super().__init__()

        self.db = LocalDB.load_state(index_path)

        self.transformed_docs: List[Document] = self.db.get_transformed_data(
            "data_transformer"
        )
        embedder = adal.Embedder(
            model_client=ModelClientType.OPENAI(),
            model_kwargs=configs["embedder"]["model_kwargs"],
        )
        # map the documents to embeddings
        self.retriever = FAISSRetriever(
            **configs["retriever"],
            embedder=embedder,
            documents=self.transformed_docs,
            document_map_func=lambda doc: doc.vector,
        )
        self.retriever_output_processors = RetrieverOutputToContextStr(deduplicate=True)

        self.generator = adal.Generator(
            prompt_kwargs={
                "task_desc_str": rag_prompt_task_desc,
            },
            model_client=OpenAIClient(),
            model_kwargs=configs["generator"],
            output_processors=JsonParser(),
        )

    def generate(self, query: str, context: Optional[str] = None) -> Any:
        if not self.generator:
            raise ValueError("Generator is not set")

        prompt_kwargs = {
            "context_str": context,
            "input_str": query,
        }
        response = self.generator(prompt_kwargs=prompt_kwargs)
        return response

    def call(self, query: str) -> Any:
        retrieved_documents = self.retriever(query)
        # fill in the document
        for i, retriever_output in enumerate(retrieved_documents):
            retrieved_documents[i].documents = [
                self.transformed_docs[doc_index]
                for doc_index in retriever_output.doc_indices
            ]

        print(f"retrieved_documents: \n {retrieved_documents}\n")
        context_str = self.retriever_output_processors(retrieved_documents)

        print(f"context_str: \n {context_str}\n")

        return self.generate(query, context=context_str), retrieved_documents

In [ ]:
# initialize rag and visualize its structure

rag = RAG(index_path="index.faiss")
rag

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db


RAG(
  (db): LocalDB(name='LocalDB', items=[Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=doc2, text='lots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonsense ...', meta_data={'title': 'Interviewing Li Yin'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_items={'data_transformer': [Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=55ea86ea-175b-4a6b-ac4e-df303cb8440f, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's pr

In [ ]:
# run RAG end to end

query = "What is Li Yin's hobby and profession?"

response, retrieved_documents = rag.call(query)
print(response)

calling the call method
retrieved_documents: 
 [RetrieverOutput(doc_indices=[0, 11], doc_scores=[0.7120000123977661, 0.6650000214576721], query="What is Li Yin's hobby and profession?", documents=[Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=090645e0-c346-428a-8419-ca21269746dc, text='textlots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonse...', meta_data={'title': 'Interviewing Li Yin'}, vector='len: 256', parent_doc_id=doc2, order=4, score=None)])]

context_str: 
  My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots

In [ ]:
# get the documents from retriever

print(retrieved_documents[0].documents)

text1, text2= retrieved_documents[0].documents[0].text, retrieved_documents[0].documents[1].text

print("rock climbing" in text1, text1)
print("software engineer" in text2, text2)


# try to manually search software engineer in the printout, you will find the key word that match the retrieval

[Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=090645e0-c346-428a-8419-ca21269746dc, text='textlots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonse...', meta_data={'title': 'Interviewing Li Yin'}, vector='len: 256', parent_doc_id=doc2, order=4, score=None)]
True My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots

# Issues and feedback

If you encounter any issues, please report them here: [GitHub Issues](https://github.com/SylphAI-Inc/LightRAG/issues).

For feedback, you can use either the [GitHub discussions](https://github.com/SylphAI-Inc/LightRAG/discussions) or [Discord](https://discord.gg/ezzszrRZvT).